In [1]:
!pip install cvxpy


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install "cvxpy[GLPK]"


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import cvxpy as cp

### Planteo del Problema

#### Conjuntos
- $G$: Conjunto de grupos de usuarios (indexados por $g$).
- $S$: Conjunto de servidores (indexados por $s$).

#### Datos
- $d_g$: Demanda de solicitudes por minuto del grupo de usuarios $g$.
- $c_{g,s}^{bw}$: Costo de ancho de banda por solicitud para el grupo de usuarios $g$ hacia el servidor $s$.
- $L_{g,s}$: Latencia promedio (ms) entre el grupo $g$ y el servidor $s$.
- $M_s$: Carga máxima de solicitudes por minuto que puede soportar el servidor $s$.

#### Parámetros
- SLA de latencia: $L_{\text{SLA}} = 400\,\text{ms}$.
- Costo por ms excedido: $c_{\text{lat}} = 0.0000001\,\text{USD/ms}$.

#### Variables de Decisión
- $x_{g,s} \geq 0$: Cantidad de solicitudes por minuto del grupo $g$ que serán atendidas por el servidor $s$.

#### Función de Costo por Latencia
Si $L_{g,s} > L_{\text{SLA}}$, el costo por latencia por solicitud es $(L_{g,s} - L_{\text{SLA}}) \cdot c_{\text{lat}}$. Si $L_{g,s} \leq L_{\text{SLA}}$, el costo es 0.

#### Función Objetivo
Minimizar el costo total por minuto:
$$
\min \sum_{g \in G}\sum_{s \in S} x_{g,s} \left( c_{g,s}^{bw} + \max(L_{g,s}-L_{\text{SLA}}, 0) \cdot c_{\text{lat}} \right)
$$

#### Restricciones
1. **Satisfacción de la Demanda por cada Grupo de Usuarios:**
$$
\sum_{s \in S} x_{g,s} = d_g \quad \forall g \in G
$$

2. **Capacidad Máxima de los Servidores:**
$$
\sum_{g \in G} x_{g,s} \leq M_s \quad \forall s \in S
$$

3. **No Negatividad:**
$$
\ x_{g,s} \geq 0 \quad \forall g \in G, \forall s \in S
$$


In [4]:
# 1. Carga de datos
demanda_df = pd.read_csv("OAL_datos_2024/demanda_usuarios.csv")
costo_bw_df = pd.read_csv("OAL_datos_2024/costo_ancho_de_banda.csv")
latencia_df = pd.read_csv("OAL_datos_2024/latencia.csv", thousands=',')
carga_max_df = pd.read_csv("OAL_datos_2024/carga_maxima_servidores.csv")
infra_actual_df = pd.read_csv("OAL_datos_2024/infraestructura_actual.csv")

In [5]:
# Extraemos listas de grupos y servidores
grupos = demanda_df["grupo de usuarios"].unique()
servidores = carga_max_df["servidor"].unique()

# Convertimos a estructuras indexadas (matrices o diccionarios)
# Aquí crearemos diccionarios para fácil acceso:
demanda = {row["grupo de usuarios"]: row["demanda (solicitudes por minuto)"] for _, row in demanda_df.iterrows()}
carga_max = {row["servidor"]: row["carga maxima (solicitudes por minuto)"] for _, row in carga_max_df.iterrows()}

# Creamos una tabla pivote para costo y latencia
costo_bw = costo_bw_df.pivot(index="grupo de usuarios", columns="servidor", values="costo del ancho de banda por solicitud (USD por solicitud)")
latencia = latencia_df.pivot(index="grupo de usuarios", columns="servidor", values="tiempo de respuesta promedio por solicitud (milisegundos)")

# Aseguramos que el orden de las filas y columnas sea coherente
costo_bw = costo_bw.reindex(index=grupos, columns=servidores)
latencia = latencia.reindex(index=grupos, columns=servidores)

In [6]:
# Definir constantes
L_SLA = 400.0
c_lat = 0.0000001

In [7]:
# 2. Definición de variables de optimización
# x_{g,s} = cantidad de solicitudes del grupo g en servidor s
x = {(g,s): cp.Variable(nonneg=True) for g in grupos for s in servidores}

In [8]:
# 3. Función objetivo
# sum_{g,s} x_{g,s} * (costo_bw[g,s] + extra_latencia*g)
latencia_excedida = (latencia - L_SLA).clip(lower=0) # Max(L_gs - 400,0)
costo_latencia = latencia_excedida * c_lat

obj = cp.sum([ x[(g,s)] * (costo_bw.loc[g,s] + costo_latencia.loc[g,s]) 
               for g in grupos for s in servidores ])

In [9]:
print("Solvers instalados: ",cp.installed_solvers())

Solvers instalados:  ['CLARABEL', 'CVXOPT', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


In [10]:
# 4. Restricciones
restricciones = []

# Demanda: sum_s x_{g,s} = d_g
for g in grupos:
    restricciones.append(cp.sum([x[(g,s)] for s in servidores]) == demanda[g])

# Capacidad: sum_g x_{g,s} <= M_s
for s in servidores:
    restricciones.append(cp.sum([x[(g,s)] for g in grupos]) <= carga_max[s])

In [11]:
# 5. Resolver el problema
prob = cp.Problem(cp.Minimize(obj), restricciones)
prob.solve(solver=cp.GLPK) # Puede usar otro solver como CBC, GLPK, etc.

print("Estado de la solución:", prob.status)
print("Costo mínimo encontrado:", prob.value)

Estado de la solución: optimal
Costo mínimo encontrado: 0.19999999999999998


In [12]:
# 6. Obtener la distribución óptima
solucion_x = pd.DataFrame([{"grupo": g, "servidor": s, "asignacion": x[(g,s)].value} 
                           for g in grupos for s in servidores])

# Filtrar las asignaciones no triviales
solucion_x = solucion_x[solucion_x["asignacion"] > 1e-9].fillna(0)

print("Distribución óptima:")
print(solucion_x)

# 7. Calcular el ahorro con respecto a la infraestructura actual
# La infraestructura actual podría traer x_actual por minuto.
# Supongamos que infraestructura_actual.csv tiene columnas: grupo, servidor, x_actual.
x_actual_df = infra_actual_df.copy()
x_actual_pivot = x_actual_df.pivot(index="grupo", columns="servidor", values="x_actual").fillna(0)

# Costo actual:
costo_actual = 0
for g in grupos:
    for s in servidores:
        req = x_actual_pivot.loc[g,s]
        lat_ex = max(latencia.loc[g,s]-L_SLA,0)
        costo_actual += req * (costo_bw.loc[g,s] + lat_ex*c_lat)

print("Costo infraestructura actual por minuto:", costo_actual)
print("Costo óptimo por minuto:", prob.value)

ahorro_por_minuto = costo_actual - prob.value
print("Ahorro por minuto:", ahorro_por_minuto)

# Asumiendo un mes de 30 días, con 1440 minutos/día:
minutos_por_mes = 30*24*60
ahorro_mensual = ahorro_por_minuto * minutos_por_mes
print("Ahorro por mes:", ahorro_mensual, "USD")

# 8. Mejora de 200ms en un servidor
# Probaremos la reducción de 200ms para cada servidor y veremos cómo impacta el costo.
# Guardamos el resultado en un diccionario para comparar.
inversion = 1000.0
mejoras = {}

for serv_mejora in servidores:
    # Creamos una copia de la latencia modificando solo serv_mejora
    latencia_mejorada = latencia.copy()
    latencia_mejorada[serv_mejora] = (latencia_mejorada[serv_mejora] - 200).clip(lower=0)
    
    # Recalcular costo latencia
    latencia_excedida_mej = (latencia_mejorada - L_SLA).clip(lower=0)
    costo_latencia_mej = latencia_excedida_mej * c_lat
    
    # Nuevo problema con la latencia mejorada
    x_mej = {(g,s): cp.Variable(nonneg=True) for g in grupos for s in servidores}
    
    obj_mej = cp.sum([ x_mej[(g,s)] * (costo_bw.loc[g,s] + costo_latencia_mej.loc[g,s]) 
                       for g in grupos for s in servidores ])
    
    restricciones_mej = []
    for g in grupos:
        restricciones_mej.append(cp.sum([x_mej[(g,s)] for s in servidores]) == demanda[g])
    for s in servidores:
        restricciones_mej.append(cp.sum([x_mej[(g,s)] for g in grupos]) <= carga_max[s])
    
    prob_mej = cp.Problem(cp.Minimize(obj_mej), restricciones_mej)
    prob_mej.solve(solver=cp.GUROBI, verbose=False)
    
    costo_mejorado = prob_mej.value
    reduccion_costo = prob.value - costo_mejorado
    
    # Tiempo de recuperación = 1000 USD / (reduccion_costo * minutos_por_mes)
    # (asumiendo que la reducción se mantiene mes a mes)
    if reduccion_costo > 0:
        tiempo_recuperacion_meses = inversion / (reduccion_costo * minutos_por_mes)
    else:
        tiempo_recuperacion_meses = np.inf
    
    mejoras[serv_mejora] = {
        "costo_mejorado": costo_mejorado,
        "reduccion_por_minuto": reduccion_costo,
        "tiempo_recuperacion_meses": tiempo_recuperacion_meses
    }

print("Resultados de la mejora de latencia (200ms menos) por servidor:")
mejora_df = pd.DataFrame.from_dict(mejoras, orient="index")
print(mejora_df)

# Elegir el servidor con el menor tiempo de recuperación
servidor_elegido = mejora_df["tiempo_recuperacion_meses"].idxmin()
print("Servidor a elegir para recuperar la inversión más rápido:", servidor_elegido)
print("Tiempo de recuperación (meses):", mejora_df.loc[servidor_elegido, "tiempo_recuperacion_meses"])


Distribución óptima:
    grupo  servidor  asignacion
0       1         1      1000.0
5       1         6      1000.0
7       2         2      1000.0
13      3         2       600.0
21      4         4       900.0
25      5         2      1200.0
30      6         1      2000.0
32      6         3      2000.0
39      7         4      2000.0
46      8         5      2000.0


KeyError: 'grupo'